# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
Turning the text into a number so the computer can understand it. In the function `text_to_ids()`, `source_text` and `target_text` are turned from words to ids.  However, we need to add the `<EOS>` word id at the end of `target_text` to help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [12]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_lines = [sentence for sentence in source_text.split('\n')]
    target_lines = [sentence + ' <EOS>' for sentence in target_text.split('\n')]
    
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_lines]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] for sentence in target_lines]
    
    return source_id_text, target_id_text

tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [13]:
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [14]:
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/Users/vaishugs/anaconda/envs/dlnd/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [63]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    inputs = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None))
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, shape=(None,), name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(tf.int32, shape=(None,), name='source_sequence_length')
    return (inputs, targets, learning_rate, keep_prob, target_sequence_length, max_target_len, source_sequence_length)

tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [64]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [65]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state

tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [66]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    training_helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length, time_major=False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer) 
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True,
                                                                maximum_iterations=max_summary_length)[0]
    return training_decoder_output
    
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [67]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_tokens = tf.tile([start_of_sequence_id], [batch_size], name='start_tokens')
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, 
                                                                end_of_sequence_id)
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, encoder_state, output_layer)
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder, impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
         
    return inference_decoder_output

tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [68]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # Decoder Embedding
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
        
    
    # Dense layer to translate the decoder's output at each time step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode"):
        training_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, 
                                         max_target_sequence_length, output_layer, keep_prob)
    
    with tf.variable_scope("decode", reuse=True):
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']

        inference_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                             end_of_sequence_id, max_target_sequence_length,
                                             target_vocab_size, output_layer, batch_size, keep_prob)
    
    return training_logits, inference_logits


    
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [69]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size, 
                                  enc_embedding_size)
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, enc_state, target_sequence_length,
                                                                       max_target_sentence_length, rnn_size,
                                                                       num_layers, target_vocab_to_int, 
                                                                       target_vocab_size,
                                                                       batch_size, keep_prob, dec_embedding_size)
                                                                       
                                                                       
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [70]:
# Number of Epochs
epochs = 20
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 40
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 30
decoding_embedding_size = 30
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.8

#display_step = 5

### Build the Graph
Build the graph using the neural network you implemented.

In [71]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [72]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/538 - Train Accuracy: 0.2626, Validation Accuracy: 0.3157, Loss: 5.0004
Epoch   0 Batch   10/538 - Train Accuracy: 0.2209, Validation Accuracy: 0.3157, Loss: 4.1129
Epoch   0 Batch   15/538 - Train Accuracy: 0.3017, Validation Accuracy: 0.3295, Loss: 3.4956
Epoch   0 Batch   20/538 - Train Accuracy: 0.3021, Validation Accuracy: 0.3535, Loss: 3.3632
Epoch   0 Batch   25/538 - Train Accuracy: 0.2877, Validation Accuracy: 0.3551, Loss: 3.3194
Epoch   0 Batch   30/538 - Train Accuracy: 0.2857, Validation Accuracy: 0.3617, Loss: 3.2464
Epoch   0 Batch   35/538 - Train Accuracy: 0.2822, Validation Accuracy: 0.3649, Loss: 3.1448
Epoch   0 Batch   40/538 - Train Accuracy: 0.3553, Validation Accuracy: 0.3643, Loss: 2.8334
Epoch   0 Batch   45/538 - Train Accuracy: 0.3322, Validation Accuracy: 0.3654, Loss: 2.8890
Epoch   0 Batch   50/538 - Train Accuracy: 0.3312, Validation Accuracy: 0.3954, Loss: 2.9184
Epoch   0 Batch   55/538 - Train Accuracy: 0.3273, Validation Accuracy

Epoch   0 Batch  450/538 - Train Accuracy: 0.4814, Validation Accuracy: 0.5036, Loss: 0.9295
Epoch   0 Batch  455/538 - Train Accuracy: 0.5057, Validation Accuracy: 0.4886, Loss: 0.8410
Epoch   0 Batch  460/538 - Train Accuracy: 0.5000, Validation Accuracy: 0.4902, Loss: 0.8854
Epoch   0 Batch  465/538 - Train Accuracy: 0.4486, Validation Accuracy: 0.5169, Loss: 0.9212
Epoch   0 Batch  470/538 - Train Accuracy: 0.4987, Validation Accuracy: 0.5199, Loss: 0.8731
Epoch   0 Batch  475/538 - Train Accuracy: 0.4980, Validation Accuracy: 0.5107, Loss: 0.8557
Epoch   0 Batch  480/538 - Train Accuracy: 0.5564, Validation Accuracy: 0.5636, Loss: 0.8572
Epoch   0 Batch  485/538 - Train Accuracy: 0.5050, Validation Accuracy: 0.5083, Loss: 0.8426
Epoch   0 Batch  490/538 - Train Accuracy: 0.4937, Validation Accuracy: 0.5462, Loss: 0.8362
Epoch   0 Batch  495/538 - Train Accuracy: 0.5605, Validation Accuracy: 0.5888, Loss: 0.8756
Epoch   0 Batch  500/538 - Train Accuracy: 0.5700, Validation Accuracy

Epoch   1 Batch  360/538 - Train Accuracy: 0.6408, Validation Accuracy: 0.6511, Loss: 0.5610
Epoch   1 Batch  365/538 - Train Accuracy: 0.6345, Validation Accuracy: 0.6422, Loss: 0.5417
Epoch   1 Batch  370/538 - Train Accuracy: 0.6297, Validation Accuracy: 0.6394, Loss: 0.5633
Epoch   1 Batch  375/538 - Train Accuracy: 0.6533, Validation Accuracy: 0.6529, Loss: 0.5041
Epoch   1 Batch  380/538 - Train Accuracy: 0.6268, Validation Accuracy: 0.6548, Loss: 0.5343
Epoch   1 Batch  385/538 - Train Accuracy: 0.6611, Validation Accuracy: 0.6477, Loss: 0.5293
Epoch   1 Batch  390/538 - Train Accuracy: 0.6724, Validation Accuracy: 0.6531, Loss: 0.5250
Epoch   1 Batch  395/538 - Train Accuracy: 0.6443, Validation Accuracy: 0.6603, Loss: 0.5492
Epoch   1 Batch  400/538 - Train Accuracy: 0.6391, Validation Accuracy: 0.6474, Loss: 0.5234
Epoch   1 Batch  405/538 - Train Accuracy: 0.6371, Validation Accuracy: 0.6534, Loss: 0.5148
Epoch   1 Batch  410/538 - Train Accuracy: 0.6287, Validation Accuracy

Epoch   2 Batch  270/538 - Train Accuracy: 0.6904, Validation Accuracy: 0.7132, Loss: 0.3669
Epoch   2 Batch  275/538 - Train Accuracy: 0.6855, Validation Accuracy: 0.7085, Loss: 0.3806
Epoch   2 Batch  280/538 - Train Accuracy: 0.7437, Validation Accuracy: 0.7152, Loss: 0.3489
Epoch   2 Batch  285/538 - Train Accuracy: 0.7178, Validation Accuracy: 0.7200, Loss: 0.3448
Epoch   2 Batch  290/538 - Train Accuracy: 0.6930, Validation Accuracy: 0.7239, Loss: 0.3524
Epoch   2 Batch  295/538 - Train Accuracy: 0.7251, Validation Accuracy: 0.7037, Loss: 0.3343
Epoch   2 Batch  300/538 - Train Accuracy: 0.6996, Validation Accuracy: 0.7221, Loss: 0.3491
Epoch   2 Batch  305/538 - Train Accuracy: 0.7221, Validation Accuracy: 0.7280, Loss: 0.3453
Epoch   2 Batch  310/538 - Train Accuracy: 0.7139, Validation Accuracy: 0.7161, Loss: 0.3618
Epoch   2 Batch  315/538 - Train Accuracy: 0.7093, Validation Accuracy: 0.7179, Loss: 0.3419
Epoch   2 Batch  320/538 - Train Accuracy: 0.7018, Validation Accuracy

Epoch   3 Batch  180/538 - Train Accuracy: 0.8006, Validation Accuracy: 0.8006, Loss: 0.2499
Epoch   3 Batch  185/538 - Train Accuracy: 0.7896, Validation Accuracy: 0.7947, Loss: 0.2402
Epoch   3 Batch  190/538 - Train Accuracy: 0.7946, Validation Accuracy: 0.7935, Loss: 0.2648
Epoch   3 Batch  195/538 - Train Accuracy: 0.8017, Validation Accuracy: 0.8001, Loss: 0.2492
Epoch   3 Batch  200/538 - Train Accuracy: 0.7969, Validation Accuracy: 0.8015, Loss: 0.2428
Epoch   3 Batch  205/538 - Train Accuracy: 0.8019, Validation Accuracy: 0.8089, Loss: 0.2371
Epoch   3 Batch  210/538 - Train Accuracy: 0.7753, Validation Accuracy: 0.8022, Loss: 0.2492
Epoch   3 Batch  215/538 - Train Accuracy: 0.7893, Validation Accuracy: 0.8004, Loss: 0.2461
Epoch   3 Batch  220/538 - Train Accuracy: 0.7535, Validation Accuracy: 0.8033, Loss: 0.2419
Epoch   3 Batch  225/538 - Train Accuracy: 0.7874, Validation Accuracy: 0.8011, Loss: 0.2332
Epoch   3 Batch  230/538 - Train Accuracy: 0.7855, Validation Accuracy

...

Epoch  13 Batch  170/538 - Train Accuracy: 0.9379, Validation Accuracy: 0.9489, Loss: 0.0331
Epoch  13 Batch  175/538 - Train Accuracy: 0.9586, Validation Accuracy: 0.9352, Loss: 0.0248
Epoch  13 Batch  180/538 - Train Accuracy: 0.9576, Validation Accuracy: 0.9482, Loss: 0.0267
Epoch  13 Batch  185/538 - Train Accuracy: 0.9748, Validation Accuracy: 0.9441, Loss: 0.0226
Epoch  13 Batch  190/538 - Train Accuracy: 0.9485, Validation Accuracy: 0.9567, Loss: 0.0354
Epoch  13 Batch  195/538 - Train Accuracy: 0.9531, Validation Accuracy: 0.9593, Loss: 0.0373
Epoch  13 Batch  200/538 - Train Accuracy: 0.9508, Validation Accuracy: 0.9574, Loss: 0.0283
Epoch  13 Batch  205/538 - Train Accuracy: 0.9492, Validation Accuracy: 0.9551, Loss: 0.0244
Epoch  13 Batch  210/538 - Train Accuracy: 0.9496, Validation Accuracy: 0.9361, Loss: 0.0338
Epoch  13 Batch  215/538 - Train Accuracy: 0.9670, Validation Accuracy: 0.9622, Loss: 0.0253
Epoch  13 Batch  220/538 - Train Accuracy: 0.9477, Validation Accuracy

Epoch  14 Batch   80/538 - Train Accuracy: 0.9475, Validation Accuracy: 0.9556, Loss: 0.0360
Epoch  14 Batch   85/538 - Train Accuracy: 0.9776, Validation Accuracy: 0.9435, Loss: 0.0216
Epoch  14 Batch   90/538 - Train Accuracy: 0.9604, Validation Accuracy: 0.9593, Loss: 0.0321
Epoch  14 Batch   95/538 - Train Accuracy: 0.9451, Validation Accuracy: 0.9428, Loss: 0.0262
Epoch  14 Batch  100/538 - Train Accuracy: 0.9637, Validation Accuracy: 0.9544, Loss: 0.0223
Epoch  14 Batch  105/538 - Train Accuracy: 0.9622, Validation Accuracy: 0.9554, Loss: 0.0201
Epoch  14 Batch  110/538 - Train Accuracy: 0.9619, Validation Accuracy: 0.9577, Loss: 0.0265
Epoch  14 Batch  115/538 - Train Accuracy: 0.9623, Validation Accuracy: 0.9453, Loss: 0.0263
Epoch  14 Batch  120/538 - Train Accuracy: 0.9646, Validation Accuracy: 0.9528, Loss: 0.0204
Epoch  14 Batch  125/538 - Train Accuracy: 0.9619, Validation Accuracy: 0.9579, Loss: 0.0287
Epoch  14 Batch  130/538 - Train Accuracy: 0.9576, Validation Accuracy

Epoch  14 Batch  525/538 - Train Accuracy: 0.9541, Validation Accuracy: 0.9437, Loss: 0.0288
Epoch  14 Batch  530/538 - Train Accuracy: 0.9361, Validation Accuracy: 0.9545, Loss: 0.0307
Epoch  14 Batch  535/538 - Train Accuracy: 0.9779, Validation Accuracy: 0.9544, Loss: 0.0301
Epoch  15 Batch    5/538 - Train Accuracy: 0.9641, Validation Accuracy: 0.9545, Loss: 0.0289
Epoch  15 Batch   10/538 - Train Accuracy: 0.9514, Validation Accuracy: 0.9506, Loss: 0.0325
Epoch  15 Batch   15/538 - Train Accuracy: 0.9567, Validation Accuracy: 0.9515, Loss: 0.0237
Epoch  15 Batch   20/538 - Train Accuracy: 0.9617, Validation Accuracy: 0.9501, Loss: 0.0262
Epoch  15 Batch   25/538 - Train Accuracy: 0.9537, Validation Accuracy: 0.9549, Loss: 0.0251
Epoch  15 Batch   30/538 - Train Accuracy: 0.9666, Validation Accuracy: 0.9558, Loss: 0.0248
Epoch  15 Batch   35/538 - Train Accuracy: 0.9512, Validation Accuracy: 0.9600, Loss: 0.0255
Epoch  15 Batch   40/538 - Train Accuracy: 0.9679, Validation Accuracy

Epoch  15 Batch  435/538 - Train Accuracy: 0.9547, Validation Accuracy: 0.9624, Loss: 0.0305
Epoch  15 Batch  440/538 - Train Accuracy: 0.9652, Validation Accuracy: 0.9656, Loss: 0.0231
Epoch  15 Batch  445/538 - Train Accuracy: 0.9561, Validation Accuracy: 0.9519, Loss: 0.0224
Epoch  15 Batch  450/538 - Train Accuracy: 0.9369, Validation Accuracy: 0.9553, Loss: 0.0347
Epoch  15 Batch  455/538 - Train Accuracy: 0.9705, Validation Accuracy: 0.9581, Loss: 0.0231
Epoch  15 Batch  460/538 - Train Accuracy: 0.9630, Validation Accuracy: 0.9593, Loss: 0.0272
Epoch  15 Batch  465/538 - Train Accuracy: 0.9646, Validation Accuracy: 0.9618, Loss: 0.0257
Epoch  15 Batch  470/538 - Train Accuracy: 0.9715, Validation Accuracy: 0.9513, Loss: 0.0251
Epoch  15 Batch  475/538 - Train Accuracy: 0.9652, Validation Accuracy: 0.9482, Loss: 0.0210
Epoch  15 Batch  480/538 - Train Accuracy: 0.9544, Validation Accuracy: 0.9572, Loss: 0.0246


KeyboardInterrupt: 

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [ ]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    sentence = sentence.lower()
    words = sentence.split()
    word_id_list = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in words]
    return word_id_list


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

## Translate
This will translate `translate_sentence` from English to French.

In [ ]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.